# Erstes Neuronales Netzwerk

Bevor wir das Netzwerk trainieren, müssen wir als Erstes unsere Libaries und Funktionen laden.


In [ ]:
import numpy as np
from matplotlib import pyplot as plt # neben Numpy laden wir auch eine Funktion der Libary matplotlib -> sie ermöglicht es Graphen in Python zu erstellen
%matplotlib inline 

### Aktiverungsfunktionen


In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x)) # np.exp() = e^()

def softmax(x, axis):
    return np.exp(x) / np.sum(np.exp(x), axis=axis, keepdims=True)

### One-hot encoding

Wir wollen die Labels nicht als einfache Zahl, sondern als Vektor aus Nullen und Einsen haben. Siehe unten:<br>
<center>
0  =  [1 0 0 0 0 0 0 0 0 0]<br>
1  =  [0 1 0 0 0 0 0 0 0 0]<br>
2  =  [0 0 1 0 0 0 0 0 0 0]<br>
3  =  [0 0 0 1 0 0 0 0 0 0]<br>
4  =  [0 0 0 0 1 0 0 0 0 0]<br>
5  =  [0 0 0 0 0 1 0 0 0 0]<br>
usw. <br>
9  =  [0 0 0 0 0 0 0 0 0 1]<br>
</center>

Die Funktion `one-hot` macht genau das.

In [ ]:
def one_hot(x):
    """Die Labels der Bilder müssen noch in Vektoren von Länge 10 codiert werden"""
    dod = len(set(x)) # Checkt wie viele verschieden Ziffern es im Datennsatz gibt
    target = np.zeros([x.shape[0], dod]) # Eine Matrix aus Nullen wird erstellt
    for i in range(x.shape[0]): # Der for-loop setzt eine 1 in die Matrix abhängig davon welches Label das Bild hat
        target[i, x[i]] = 1

    return target

In [ ]:
# Skaliert die originalen Bilder, die Werte zwischen 0 und 255 haben können, zu Bildern mit Werten zwischen 0 und 1.
# Neuronale Netzwerke können besser trainiert werden
def min_max(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

### Daten einlesen

Wir können mit Numpy ohne Probleme die benötigten Daten einlesen. Die Trainingsdaten `mnist_train.csv` enhalten Bilder und deren Labels. Die Bilder sind schon von einer Matrix in einen Vektor transformiert worden.<br>
<center>
<img src="https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/73_blog_image_1.png" style="width: 800px;">
</center>

Der Datensatz kann auch extern angeschaut werden. Dafür klicken Sie einfach in der linke Spalte (je nach Jupyter Version) auf den Ordner `Data` und dannach auf die Datei `mnist_train.csv`. Die erste Spalte des jeweiligen Datensatzes enthält die Labels. In jeder Reihe ist ein Bild, zudem ist jeder Spalte (bis auf der ersten) ein Pixeln zugeordnet.

Mit `np.genfromtxt()` lassen sich aus .txt-Datein Arrays in Python erstellen. 

In [ ]:
train_data = np.genfromtxt('Data/mnist_train.csv', delimiter=',', skip_header =False) #genfromtxt liest .txt Datein, mit delimiter ="," können auch .csv (comma seperated values) Datein einglesen werden  
test_data=np.genfromtxt('Data/mnist_test.csv', delimiter=',', skip_header =False) # hier lesen wir die Test Daten ein

# Nach dem wir die Daten einglesen haben, trennen wir die Daten noch in Bilder und Labels
# Des Weiteren konvertieren wir Labels von Float zu Integer mit .astype(int)
train_labels=train_data[:,0].astype(int) 
train_images = train_data[:,1:]

test_labels=test_data[:,0].astype(int)
test_images = test_data[:,1:]

Wir haben jetzt den Trainingsdatensatz eingelesen. Sie können mit `train_images.shape` sehen, dass die Variable `train_images` eine 60000 x 784 Matrix ist, also 60000 Reihen und 784 Spalten hat. Jede Reihe ist ein Bild und jede Spalte ist ein Pixel. Die orginalen Bilder waren 28 x 28 Pixel groß. Im nächsten Schritt benutzen wir die *Funktion* `one_hot()` um die Labels richtig zu codieren.

In [ ]:
train_targets=one_hot(train_labels)
test_targets = one_hot(test_labels)

train_targets[:5,:] # die Labels der ersten fünf Bilder

Als letzten Schritt müssen wir noch den `min_max` scaler benutzen, um die Pixelwerte zwichen Nulls und Eins zu skalieren.

In [ ]:
train_images = min_max(train_images)
test_images = min_max(test_images)

In der folgenden Zelle können Sie ein Beispielbild sehen. Die Funktion `.reshape([28,28])` transformiert das Bild in seine ursprüngliche Version zurück. So kann man das Bild auch wiklich erkennen.

In [ ]:
plt.imshow(train_images[0].reshape([28, 28]), cmap="gray")
print("Correct Label: %s" % train_labels[0])

### Das Model


**Weights Initalisieren** <br>
Als Erstes müssen die Weight und Bias Matrizen/Vektoren mit den richtigen Dimensionen erstellt werden. Dafür wird eine Funktion geschrieben. In ihr muss man die Größe der jeweiligen Layer (`input_size`, `hidden_size`, `output_size`) festlegen. Mit diesen können Sie die Weightmatrizen erstellen. Während die `b`- Vektoren mit Nullen gefüllt werden können, müssen die Weightmatrizen mit zufälligen kleinen Zahl initialisiert werden.

Die Funktion `np.random.randn(number_of_rows, number_of_columns)` setzt zufällige Werte zwischen minus Eins und plus Eins in die Matrix ein. Zusätzlich wird der Term`* np.sqrt(2/layer_size)` hinzugefügt. Dieser verkleinert die Werte noch weiter und soll damit ein besseres Trainieren des Netzwerkes garantieren. 

Im folgenden Code sollen Sie die richtigen Matrixgrößen (Anzahl der Reihen/Spalten) eintragen. Die Inputmatrix `X` hat 60000 Reihen und 784 Spalten und bei einer Matrixmultiplikation müssen die Anzahl der Spalten der ersten Matrix mit der Anzahl der Reihen der zweiten Matrix übereinstimmen.
Denkt Sie auch daran, dass wir das Transpose der Weightmatrix benutzen werden.

Den Bias `b` wird mit Nullen initialisiert. `np.zeros(500)` würde eine Vektor der Länge 500 mit Nullen füllen.
Auch brauchen wir nicht die tatsächlien Zahlen, sondern nur die Namen der Inputvariablen nennen,  also nicht 784 sondern `input_size`.
Die  tatsächlichen Werte können dann bei der eigentlichen Benutzung der Funktion definiert werden.

In [ ]:
# Funktion zur intialisierung der Weights
def init_weights(input_size, hidden_size, output_size):
    # Zwei leere Listen mit Länge zwei
    b = [0] * 2
    W = [0] * 2

    # hier werden die Weights W mit kleinen, zufälligen Gewichten initialisiert
    
    W[0] = np.random.randn(        ,         ) * np.sqrt(2 / input_size)  # HIER MÜSSEN DIE RICHTIGEN GRÖßEN ANGEGBEN WERDEN
    W[1] = np.random.randn(        ,         ) * np.sqrt(2 / hidden_size) # HIER MÜSSEN DIE RICHTIGEN GRÖßEN ANGEGBEN WERDEN

    # der bias kann 0 sein
    b[0] = np.zeros(       ) # HIER MÜSSEN DIE RICHTIGEN GRÖßEN ANGEGBEN WERDEN
    b[1] = np.zeros(       ) # HIER MÜSSEN DIE RICHTIGEN GRÖßEN ANGEGBEN WERDEN


    return W, b

Die Weights können jetzt initialisiert werden: 

Die Inputgröße ist vorgegeben, da 784 Pixel in jedem Bild sind. Auch die Outputgröße ist vorgegeben, da das Netzweerk zwischen 10 verschiedene Ziffern unterscheiden muss.
Der einzige Wert, den Sie selber bestimmen können, ist die Größe der Hidden-Layer. Es gibt keine einfache Regel für die Größe, aber wenn sie zu klein ist, kann das die Genauigkeit des Neuronalen Netzwerkes verringern. Wenn sie zu groß ist, trainiert das Netzwerk zu langsam und wird eventuell instabil. Auch das kann die Genauigkeit beeinflussen. Probieren Sie am Anfang Werte wie 100 oder 200 aus. 

Grundsätzliche werden Zahlen wie die Größe der Hidden-Layer oder die Lernrate, die Sie frei wählen können, als **Hyperparameter** bezeichnet. 

Mit dieser Funktion können jetzt die Weight initialisiert werden. Die Funktion gibt Ihnen zwei Listen `W` und `b` aus. Die beiden Listen enhalten jeweils zwei Matrizen/Vektoren, einmal für die erste und einmal für die zweite Transformation.

In [ ]:
W, b = init_weights(input_size=784, hidden_size=200,output_size= 10)

Wenn Sie die Weights richtig initialisiert haben, sollte `W[0].shape` `(200,784)` sein.

In [ ]:
W[0].shape

**Forward Pass** <br>

Nachdem die Weights erstellt worden sind, können Sie den Forwardpass durch das Netzwerk durchführen. Die Bilder werden durch das Netzwerk geschickt, welches am Ende diese Bilder klassifiziert.
Die Funktion `forward_pass()` macht genau das. Ihr Input ist:
* `W`: die Liste der Weight Matrizen
* `b`: die Liste der Bias Vektoren
* `X`: die Inputmatrix der Bilder 


Als Erstes wird $z_1 = xW_1^T+b_1$ berechnet. <br>
Dannach $a_1= sigmoid(z_1)$ <br>
<br>
Somit sind die Activations für die Hiddenlayer berechnet. Um die Klassifzierung machen zu können, müssen die Werte noch ein zweites mal transformiert werden:<br>
$\hat{y} = softmax(a_1W_2^T+b_2)$

Die Funktion gibt am Ende drei Variablen aus:
* `z_1`: die Werte der ersten linearen Transformation
* `a_1`: die Werte nach der ersten Aktivierungsfunktion
* `y_hat`: die Werte der Outputlayer 

<br>
<br>

*Denkt daran, dass die Weights der ersten Layer nicht in `W[1]`, sondern in `W[0]` gespeichert sind, da die Indizierung bei Null und nicht bei Eins anfängt.* 

In [ ]:
def forward_pass(W, b, X):
    
    z_1 = # CODE UM Z_1 AUZURECHNEN
    a_1 = # CODE UM A_1 AUSZURECHNEN
    z_2 = # CODE UM Z_2 AUSZURECHNEN
    y_hat = #CODE UM Y_HAT AUSZURECHNEN
    return z_1, a_1, y_hat

**Loss Funktion**

Nach dem Forward Pass wird der Loss berechnet. Dieser misst, wie gut oder schlecht das Model die Zahlen klassifizieren kann.

Hierfür werden nur die Werte der Output Layer (`y_hat`) und die tatsächlichen Werte (`y`) gebraucht. Die wahren Werte wurden schon vorher mit der Funktion `one_hot` berechnet und `train_targets` genannt. 

In [ ]:
def calc_loss(y_hat, y):
    return -np.sum(np.log(y_hat) * y)

Sie können jetzt die ersten drei Funktion zusammen benutzen, um Ihre erste Klassifizierung durchzuführen.
Denkt daran, dass die `train_images` der Input für das Netzwerk sind.

In [ ]:
np.random.seed(1234) # Ein Seed wird gesetzt, damit die Ergebnisse der zufälligen Initialisierung für alle Teilnehmer:innen gleich sind
W, b = # CODE, UM DIE WEIGHTS ZU INITIALISIEREN

# Benutzen Sie die train_images als X (Input)
z_1, a_1, y_hat = # CODE FÜR DEN FORWARD PASS

# Hier berechnen Sie den Loss
calc_loss(           ,           )/y_hat.shape[0]

Der Loss ist sehr schwierig zu interpretieren. 
Deswegen können wir zusätzlich die Genaugikeit (accuracy) als Maß benutzen. Die Genauigeit ist der Prozentsatz an richtig klassifizierten Bildern
$ \frac{n_{richtig}}{n}$.

Die Genauigkeit ist ein viel einfacheres Maß um zu beurteilen, wie gut das Model funktioniert.

In [ ]:
def accuracy(true_labels,predicted):
    return (np.sum(true_labels == np.argmax(predicted, axis=1)) / predicted.shape[0]) # argmax gibt den Index des Maximalwertes wieder

In [ ]:
accuracy(train_labels,y_hat)

Momentan hat das Netzwerk eine Genauigkeit von 9,8% Prozent. 10% Genauigkeit wären zu erwarten, wenn das Netzwerk zufällig entscheiden würde.
Um besser als 9,7% zu werden, müssen Sie das Netzwerk trainieren und die Weights verändern. Dazu benutzen Sie die Back Propagation.

**Back Propagation**

Der Fehler in der Klassifizierung wird nun zurück durch das Netztwerk geschickt und mit Hilfe der Gradienten werden die Weights angepasst.

* $dZ_2 = \hat{y} - y$ 
* $dW_2 = \frac{1}{n} \cdot dZ_2^Ta_1$
* $db_2 = \frac{1}{n} \cdot \sum_{i=1}^n dZ_2$
* $dZ_1 = dZ_2W_2 \cdot a_1 \cdot (1-a_1)$
* $dW_1 = \frac{1}{n} \cdot dZ_1^TX$
* $db_1 = \frac{1}{n} \cdot\sum_{i=1}^n dZ_1$

Wie gesagt: Wie die Ableitungen mathematische Zustande kommen, sprengt den Rahmen dieses Seminar. In der tatsächlichen Anwendung gibt es Libaries, welche die Gradienten automatisch berechenen können. 

In [ ]:
def back_prop(x, z_1, a_1, y_hat, y):
    n = x.shape[0] # n ist die Anzahl der Bilder
    
    # Gradients für die Weights der zweiten Layer
    dz_2 =  # CODE UM dZ_2 AUZURECHNEN
    dW_2 =  # CODE UM dW_2 AUZURECHNEN
    db_2 = np.sum(dz_2, axis=0) / n
    
    # Gradients für die Weights der ersten Layer
    dz_1 = np.multiply(np.matmul(dz_2, W[1]), np.multiply(a_1, 1 - a_1))
    dW_1 = # CODE UM dW_1 AUZURECHNEN
    db_1 = # CODE UM db_1 AUZURECHNEN

    return [dW_1, dW_2], [db_1, db_2] # Hier werden wieder zwei Listen als Output gegeben, in jeder der Listen befinden sich jeweils die Gradienten für W_1,W_2 und b_1 und b_2

**Weights Updaten**

Im letzten Schritt werden die Weights angepasst. Dafür werden die Weights ein kleinwenig entgegen des Gradienten verschoben. 
Wie weit genau, hängt von der Lernrate `lr` ab. Je größer diese ist, umso größer werden die Schritte.
Ist die Lernrate zu klein dauert das Trainieren eventuell zu lange, oder das Netzwerk bleibt in einem lokalen Minimum hängen. Ist die Lernrate zu groß, springt der Loss des  Netzwerkes zu viel und dementsprechend kann kein optimale Performance garantiert werden.
Die Lernrate gehört, wie die Größe der Hidden Layer, zu den Hyperparametern.
<center>
<img src="https://sebastianraschka.com/images/blog/2015/singlelayer_neural_networks_files/perceptron_learning_rate.png" style="width: 800px;">
<h8><center>Source: Sebastian Raschka, https://sebastianraschka.com</center></h8>
</center>






In [ ]:
def update(W, b, grad_W, grad_b, lr=0.0001):
    W[0] = # CODE FÜR W[0]
    W[1] = # CODE FÜR W[1]
    b[0] = # CODE FÜR b[0]
    b[1] = # CODE FÜR b[1]

    return W, b # die Funktion gibt die neuen Weights und Biases aus (2 Listen)

## Putting it all together

Jetzt können Sie alles zusammenfügen. Sie initalisieren erst die Weights, dann werden die Inputs durch das Netzwerk geschickt und der Loss wird berechnet.  Anschließend wird der Loss wieder zurück durch das Netzwerk geführt und die Gradienten berechnet. Die Gradienten werden benutzt die  Weights zu updaten. Für die Lernrate gebrauchen Sie 0,1.

In [ ]:
# 1.WEIGHTS INITALISIEREN
np.random.seed(1234) 
W, b = init_weights(input_size=784, hidden_size=200,output_size= 10) 

# 2.FORWARD PROPAGATION
z_1, a_1, y_hat = forward_pass(W,b,train_images)

# 3.LOSS BERECHNEN
print("Loss beim ersten Durchlauf: ",calc_loss(y_hat,train_targets)/y_hat.shape[0], "Genauigkeit beim ersten Durchlauf", accuracy(train_labels, y_hat) )

# 4. BACKPROPAGATION
grad_W, grad_b = # CODE FÜR DIE BACKPROPAGATION
# 5. WEIGHTS UPDATEN
W, b = # WEIGHT UPDATE

Der Loss und die Genauigkeit sind noch gleich. Erst wenn Sie erneut den Input durch das Netzwerk schicken, können Sie den Effekt der veränderten Weights sehen.

In [ ]:
z_1, a_1, y_hat = forward_pass(W,b,train_images)

# jetzt berechnen Sie den Loss
print("Loss beim zweiten Durchlauf: ",calc_loss(y_hat,train_targets)/y_hat.shape[0], "Genauigkeit beim zweiten Durchlauf", accuracy(train_labels, y_hat) )

Tatsächlich verringert sich der Loss, aber die Genauigkeit hat sich auch verringert. Das kann während des Trainings passieren, ist aber nicht weiter schlimm.  Der Effekt des Trainings wird oft erst nach mehreren Epochs sichtbar. Sie können den Schritt des Trainierens einfach wiederholen.
Um dies effizienter zumachen, können Sie einfach einen `for-loop` benutzen:

In [ ]:
# 1.WEIGHTS INITALISIEREN
np.random.seed(1234)
W, b = init_weights(784, 200, 10)

EPOCHS= 50 # wie oft der Datensatz durch das Netzwerk geführt wird
for i in range(EPOCHS):
    
    # 2. FORWARD PROPAGATON
    z_1, a_1, y_hat = forward_pass(W, b,train_images)
    
    # 3. LOSS BERECHNEN
    loss = calc_loss(y_hat, train_targets) / y_hat.shape[0]
    acc = accuracy(train_labels, y_hat)
    
    print(
        "Training Loss: %.2f Training Accuracy: %.2f"
        % (loss, acc)
    )
    
    # 4. BACK PROPAGATION
    grad_W, grad_b = back_prop(train_images, z_1, a_1, y_hat, train_targets)
    
    # 5. WEIGHTS UPDATEN
    W, b = update(W, b, grad_W, grad_b, lr = 0.1)

Sie sehen schon eine Verbesserung und kommen auf eine Genaugigkeit von 69%. Allerdings dauert das Trainieren sehr lange. Eine größere Lernrate sollte das Training schneller machen. Probieren Sie eine Lernrate von 0,3 aus.

In [ ]:
np.random.seed(1234)
W, b = init_weights(784, 200, 10)
loss= []
EPOCHS= 50 # wie oft der Datensatz durch das Netzwerk geführt wird
for i in range(EPOCHS):
    z_1, a_1, y_hat = forward_pass(W, b,train_images)
    loss= calc_loss(y_hat, train_targets) / y_hat.shape[0]
    acc = accuracy(train_labels, y_hat)
    print(
        "Training Loss: %.2f Training Accuracy: %.2f"
        % (loss, acc)
    )
    
    # wir machen die backpropagation
    grad_W, grad_b = back_prop(train_images, z_1, a_1, y_hat, train_targets)
    # und mit den Gradienten updaten wir die Weights
    W, b = update(W, b, grad_W, grad_b, lr = )
    

Mit einer Lernrate von 0.3 erreichen Sie nach 50 Epochs eine Genauigkeit von 81%. Sie können das Netzwerk noch weiter verbessern indem Sie  noch  länger trainieren. Was müssten Sie am Code ändern, damit für 25 weitere Epochs trainiert wird **ohne, dass das Netzwerk ganz neu trainiert werden muss**?

In [ ]:
np.random.seed(1234)
W, b = init_weights(784, 200, 10)
loss= []
EPOCHS= 25 # wie oft der Datensatz durch das Netzwerk geführt wird
for i in range(EPOCHS):
    z_1, a_1, y_hat = forward_pass(W, b,train_images)
    loss= calc_loss(y_hat, train_targets) / y_hat.shape[0]
    acc = accuracy(train_labels, y_hat)
    print(
        "Training Loss: %.2f Training Accuracy: %.2f"
        % (loss, acc)
    )
    
    # wir machen die backpropagation
    grad_W, grad_b = back_prop(train_images, z_1, a_1, y_hat, train_targets)
    # und mit den Gradienten updaten wir die Weights
    W, b = update(W, b, grad_W, grad_b, lr = 0.3)

Durch weiteres Training haben Sie das Netzwerk um 3% Genauigkeit verbessern können. Grundsätzlich haben Sie noch die Möglichkeit mit der Größe der Hidden Layer etwas zu verändern. 

Bevor wir das machen, können Sie erst einmal schauen, wie gut unser Modell funktioniert für Bilder, die es noch nicht gesehen hat. Die Rede ist von dem Testdatendatz. Dazu speisen sie den Testdatensatz in das trainierte Netzwerk ein:

In [ ]:
_, _, test_y_hat = forward_pass(W, b, test_images) # durch das _,_,test_y_hat werden z_1 und a_1 nicht mit ausgegeben, da wir diese nicht brauchen
accuracy(test_labels, test_y_hat)   

Die Genauigkeit auf dem Testdatensatz beträgt auch 85%. Man kann sagen, dass 85% der Bilder richtig erkannt wurden.
Es ist unüblich, dass Netzwerke besser für den Testdatensatz funktioniert. Dies ist ein Hinweis darauf, dass Sie das Modell nicht lange genug trainiert haben.

Als nächstes können Sie sich anschauen, bei welchen Bilder das Netzwerk am meisten Probleme hat.
Der Code in der nächsten Zelle, sortiert die fasch identifizierten Bilder nach ihre Wahrscheinlichkeit (Dieser Code ist nicht unbedingt leicht zu verstehen, ist aber für das Verständnis von Neuronalen Netzwerken auch nicht essentiell):

In [ ]:
falsche_klassifizierung = np.where(test_labels != np.argmax(test_y_hat, axis=1))[0]# welche Bilder wurde falsch klassifiziert
len(falsche_klassifizierung) # soviele Bilder wurden falsch klassifiziert

In [ ]:
# Hier werden de Wahrscheinlichkeiten gesammelt die das Model dem Bild zugeorndet hat in der richtige Kategorie zu sein
probs = [] 
for image in falsche_klassifizierung:
    probs.append(test_y_hat[image,test_labels[image]])

In [ ]:
# Wir sortieren die Bilder Anhand der Wahrscheinlichkeiten, je kleiner die Wahrscheinlichkeit desto sicherer war das Model das das Bild nicht in der richtige Katehorie ist
falsche_klassifizierung=falsche_klassifizierung[np.argsort(probs)]

In [ ]:
# so sehen 10 Bilder aus die falsch klassifiziert werden
for i in range(10):
    plt.imshow(test_images[falsche_klassifizierung[i]].reshape([28, 28]), cmap="gray")
    plt.show()
    print(
        "Predicted Label: %s, Correct Label %s"
        % (
            np.argmax(test_y_hat, axis=1)[falsche_klassifizierung[i]],
            test_labels[falsche_klassifizierung[i]],
        )
    )

Bei manchen Bilder können Sie klar sehen, warum diese falsch kategorisiert wurden. Bei anderen jedoch ist es für ein menschliches Auge ganz einfach die richtige Ziffer zuerkennen, trotzdem hat das Netzwerk damit Probleme.

Das war es für den ersten Tag. Wenn Sie wollen, könnem Sie noch mit den Hyperparameter spielen um Ihr Netzwerk zu verbessern.